In [1]:
import pandas as pd
import math
import numpy as np
from scipy.optimize import minimize

In [2]:
xls = pd.ExcelFile('Assignment 2 BUSI712.xlsx')
data = pd.read_excel(xls, 'Test Data', header = 1)
data

/Users/huailintang/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,Segment,m_s,x_s
0,1,300,2
1,2,273,1
2,3,499,0
3,4,428,1
4,5,433,1
5,6,296,0
6,7,451,3
7,8,104,0
8,9,426,0
9,10,490,4


In [3]:
def nbetabinomialLL(par, m_s, x_s):
    a = par[0]
    b = par[1]
    ldenom = math.exp(math.lgamma(a)+math.lgamma(b)-math.lgamma(a+b))
    llsum = 0
    for t in range(len(data)):
        llsum += np.log(math.comb(m_s[t], x_s[t])*math.exp(math.lgamma(a+x_s[t])+math.lgamma(b+m_s[t]-x_s[t])-math.lgamma(a+b+m_s[t]))/ldenom)    
    return -llsum

In [4]:
bnds = ((0.0001,None),(0.0001,None)) 

In [5]:
res = minimize(nbetabinomialLL, (1,1), (data['m_s'], data['x_s']), method="nelder-mead",
               options={'xatol': 1e-08, 'disp': True},bounds=bnds)

Optimization terminated successfully.
         Current function value: 73.850394
         Iterations: 104
         Function evaluations: 223


In [6]:
print('MLE for a,b is', res.x, 'and LL is',-nbetabinomialLL(res.x,data['m_s'], data['x_s']))

MLE for a,b is [  1.6131702  362.81987103] and LL is -73.85039380829467


In [7]:
ldenom = math.exp(math.lgamma(res.x[0])+math.lgamma(res.x[1])-math.lgamma(res.x[0]+res.x[1]))

In [8]:
predict = []
for t in range(len(data)):
    prediction = math.comb(data['m_s'][t], data['x_s'][t])*math.exp(math.lgamma(res.x[0]+data['x_s'][t])+math.lgamma(res.x[1]+data['m_s'][t]-data['x_s'][t])-math.lgamma(res.x[0]+res.x[1]+data['m_s'][t]))/ldenom
    predict.append(prediction)

In [9]:
data['beta_bino_predict'] = predict

In [10]:
data['simple_predict'] = data['x_s'] / data['m_s']

In [11]:
RR = (2000/10000)/100

In [12]:
expectation = (res.x[0] + data['x_s']) / (res.x[0] + res.x[1] + data['m_s'])

In [13]:
data['expectation'] = expectation

In [14]:
data['target'] = expectation >= RR

In [15]:
target_data = data[data['target'] == True]
target_data

,Segment,m_s,x_s,beta_bino_predict,simple_predict,expectation,target
0,1,300,2,0.163637,0.006667,0.005438,True
1,2,273,1,0.280807,0.003663,0.004100,True
3,4,428,1,0.248909,0.002336,0.003298,True
4,5,433,1,0.247702,0.002309,0.003277,True
5,6,296,0,0.382237,0.000000,0.002443,True
6,7,451,3,0.117546,0.006652,0.005657,True
7,8,104,0,0.666127,0.000000,0.003444,True
8,9,426,0,0.285903,0.000000,0.002041,True
9,10,490,4,0.080396,0.008163,0.006569,True
10,11,234,3,0.068396,0.012821,0.007709,True


In [16]:
num = sum(target_data['m_s']) * 100 # Multiply by 100 because the data only consists of 1% of customers

In [17]:
cost = num * (2000/10000)
cost

254160.0